# Using typedspark in Databricks notebooks
You can dynamically load a `DataSet` and its corresponding `Schema` from a table in Databricks. To illustrate this, let us first make a temporary table that we can load later.

In [2]:
import warnings
from pyspark.sql import SparkSession 
warnings.filterwarnings('ignore')
spark = SparkSession.Builder().getOrCreate()

In [3]:
import pandas as pd

(
    spark.createDataFrame(
        pd.DataFrame(
            dict(
                name=["Jack", "John", "Jane"],
                age=[20, 30, 40],
            )
        )
    )
    .createOrReplaceTempView("person_table")
)


We can now load these data using `load_table()`. Note that the `Schema` is inferred: it doesn't need to have been serialized using `typedspark`.

In [4]:
from typedspark import load_table

df, Person = load_table(spark, "person_table")

You can now use `df` and `Person` just like you would in your IDE. One of the advantages is that this provides auto-complete on column names. No more looking at `df.columns` every minute to remember the column names!

In [5]:
(
    df
    .filter(Person.age > 25)
    .show()
)

+----+---+
|name|age|
+----+---+
|John| 30|
|Jane| 40|
+----+---+



Of note, `load_table()` automatically runs `register_schema_to_dataset()` on the resulting `DataSet` and `Schema`, hence resolving potential column disambiguities.

A second situation in which `load_table()` is helpful is when you want to add schemas to existing tables. Simply run:

In [6]:
Person


from pyspark.sql.types import LongType, StringType

from typedspark import Column, Schema


class DynamicallyLoadedSchema(Schema):
    name: Column[StringType]
    age: Column[LongType]

To also generate documentation, run:

In [7]:
Person.print_schema(schema_name="Person", include_documentation=True)

from typing import Annotated

from pyspark.sql.types import LongType, StringType

from typedspark import Column, ColumnMeta, Schema


class Person(Schema):
    """Add documentation here."""

    name: Annotated[Column[StringType], ColumnMeta(comment="")]
    age: Annotated[Column[LongType], ColumnMeta(comment="")]

